In [1]:
"""Script to pre-process AML transaction data to be used in training and inference."""
import os
import argparse
import logging
from datetime import datetime

import pandas as pd
import numpy as np
from icecream import ic

# Setup logging
logger = logging.getLogger(__name__)
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)

In [18]:
df = pd.read_csv('/mnt/data/ibm-transactions-for-anti-money-laundering-aml/HI-Large_Trans.csv')
# print the histogram of Amount Recieved column
print(df['Amount Received'].hist(bins=10000, color='blue', edgecolor='black'))
# print min and max
print(df['Amount Received'].min())
print(df['Amount Received'].max())

: 

In [ ]:
# Group by 'Account' and aggregate 'From Bank'
grouped = df.groupby('Account')['From Bank'].unique()

# Find accounts with multiple banks
duplicated_accounts = grouped[grouped.apply(lambda x: len(x) > 1)]

print(duplicated_accounts)

Account
80C746490        [20703, 19200]
81A27EB30      [160226, 264674]
81F71BE10       [60269, 157344]
82954CE40      [144465, 143318]
83212BF10    [2125527, 1135003]
83214F0E0    [2129483, 2133788]
8347C8F30    [2142416, 2142968]
83510AC20       [51969, 243938]
83B7523F0     [2121060, 298409]
83FFB8A70     [172737, 1166034]
84ADF03D0       [164028, 38633]
851912640      [246267, 238428]
Name: From Bank, dtype: object
